In [1]:

import os
import sys, glob
import awkward as ak
import numpy as np
import torch
import argparse
from fcc_study.pNN.training.preprocessing_datasetClasses import (
    CustomDataset,
    getData,
)
import json
import importlib
import pickle
from torch.utils.data import DataLoader
from tqdm import tqdm
import copy
from importlib import import_module

In [2]:
events = ak.from_parquet("/vols/cms/emc21/FCC/FCC-Study/runs/main_run/run7/data/test/awkward/mH85_mA125_h2h2ll.parquet")

branches = ['Zcand_m',
 'Zcand_pt',
 'Zcand_pz',
 'Zcand_p',
 'Zcand_povere',
 'Zcand_e',
 'Zcand_costheta',
 'Zcand_recoil_m',
 'lep1_pt',
 'lep1_eta',
 'lep1_e',
 'lep1_charge',
 'lep2_pt',
 'lep2_eta',
 'lep2_e',
 'lep2_charge',
 'lep_chargeprod',
 'cosDphiLep',
 'cosThetaStar',
 'cosThetaR',
 'n_jets',
 'MET_e',
 'MET_pt',
 'MET_eta',
 'MET_phi',
 'n_photons',
 'n_muons',
 'n_electrons']

import pickle
with open('/vols/cms/emc21/FCC/FCC-Study/runs/main_run/run7/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('/vols/cms/emc21/FCC/FCC-Study/runs/main_run/run7/mass_scaler.pkl', 'rb') as f:
    mass_scaler = pickle.load(f)

In [6]:
for branch in branches:
    field = events[branch]
    print(f"{branch: <25}", field)

    if ak.all(field == field[0]):
        print(f"Useless branch!")

Zcand_m                   [11.6, 16.6, 13.1, 9.22, 36.4, 27.1, ..., 33.4, 19.5, 22.4, 19.7, 4.22, 14.9]
Zcand_pt                  [27.7, 39.2, 6.73, 32.4, 28.5, 7.45, ..., 21.2, 31.9, 14.4, 44.2, 8.59, 23.3]
Zcand_pz                  [21.2, 23.8, 24.1, -2.28, -12.2, 9.29, ..., 6.38, -8.41, 16.9, 10.5, 20, 21.2]
Zcand_p                   [34.9, 45.9, 25, 32.5, 31, 11.9, 21.9, ..., 22.1, 33, 22.2, 45.4, 21.7, 31.5]
Zcand_povere              [0.949, 0.94, 0.886, 0.962, 0.648, ..., 0.861, 0.704, 0.918, 0.982, 0.904]
Zcand_e                   [36.7, 48.8, 28.3, 33.8, 47.8, 29.6, ..., 40.1, 38.3, 31.5, 49.5, 22.1, 34.8]
Zcand_costheta            [0.608, 0.519, 0.963, -0.0701, -0.393, ..., -0.255, 0.761, 0.231, 0.919, 0.673]
Zcand_recoil_m            [200, 186, 210, 204, 190, 210, 210, 204, ..., 216, 199, 199, 207, 185, 217, 203]
lep1_pt                   [19.5, 35, 7.49, 22.4, 20.7, 15.7, 20.2, ..., 18.1, 32.7, 10.3, 27, 5.38, 13.5]
lep1_eta                  [0.395, 0.375, 1.03, -0.0762, 0.4

In [3]:
data_feats = events[branches]
data_feats

data_ms = events[['mH', 'mA']]

data_ms

data_w = np.ones(len(data_feats))
data_Y = np.ones(len(data_feats))
data_wscaled = np.ones(len(data_feats))

In [4]:
def convertToNumpy(events: ak.Array, branches: list) -> np.array:
    # if only 1 branch then don't do the view stuff below, as that returns
    # garbage
    if len(branches) == 1:
        return ak.to_numpy(events[branches[0]])

    numpy_data = (
        ak.to_numpy(events[branches]).view("<f4").reshape(-1, len(branches))
    )
    return numpy_data

# data_feats = convertToNumpy(data_feats, branches)
# data_ms = convertToNumpy(data_ms, ['mH', 'mA'])

In [5]:
def applyScaler(evs, scaler, branches):
    numpy_data = convertToNumpy(evs, branches)
    trans_numpy_data = scaler.transform(numpy_data)
    for i, br in enumerate(branches):
        evs[br] = trans_numpy_data[:, i]

    return evs


# Scale the features
events = applyScaler(events, scaler, branches)
events = applyScaler(events, mass_scaler, ['mH', 'mA'])
# data_feats = feat_scaler.transform(data_feats)
# data_ms = mass_scaler.transform(data_ms)

# Now put into a helper class
data_dataset = CustomDataset(events, branches, scaler, mass_scaler)

Initialising CustomDataset


In [6]:
data_dataset